## Are EATCH (energy above the convex hull) values from Materials Project correct?

In [4]:
import json
import gzip
from schema import Compound, load_mp_pairs
from pymatgen.analysis.phase_diagram import PhaseDiagram
from pymatgen.core.composition import Composition
from pymatgen.entries.computed_entries import ComputedEntry
from tqdm import tqdm


In [5]:

# load mp data
with gzip.open("mp.json.gz", 'rt', encoding='UTF-8') as zipfile:
    mpdata = json.load(zipfile)

In [6]:
# for a compound A, find other compounds in MP whose elements form a subset of A's elements
pairs = load_mp_pairs(mpdata)

create mpid2chemsys...


100%|██████████| 126335/126335 [00:00<00:00, 323454.72it/s]


create chemsys2subsets...


100%|██████████| 43888/43888 [02:45<00:00, 265.74it/s]


create pairs...


100%|██████████| 126335/126335 [00:06<00:00, 18956.16it/s]


In [8]:
# wrapper for pymatgen EATCH calculator
def find_pmgehull(
        reactant: Compound, products: list[Compound],
):
    c_e = ComputedEntry(Composition(reactant.normalized_formula), reactant.formation_energy_per_atom,
                        entry_id=reactant.mpid)
    cp_es = []
    for cp in products:
        cp_es.append(ComputedEntry(Composition(cp.normalized_formula), cp.formation_energy_per_atom, entry_id=cp.mpid))
    pdEntries = [c_e, ] + cp_es
    pd = PhaseDiagram(pdEntries)
    decomp, ehull = pd.get_decomp_and_e_above_hull(c_e)
    return ehull

In [32]:
# how many inconsistent hull values?
inconsistent_pairs = []
for pair in tqdm(pairs[:500]):  # let's look at the first 500
    pmg_ehull = find_pmgehull(*pair)
    mp_ehull = pair[0].e_above_hull
    try:
        assert isinstance(pmg_ehull, float)
        assert isinstance(mp_ehull, float)
    except AssertionError:
        continue
    if abs(pmg_ehull - mp_ehull) > 1e-1:
        inconsistent_pairs.append(pair)

100%|██████████| 500/500 [00:17<00:00, 29.36it/s]


In [39]:
def printout_inconsistency(pair):
    compound, competing_phases = pair
    pmg_ehull = find_pmgehull(*pair)
    mp_ehull = compound.e_above_hull
    print("Compound: {}; Competing phases: {}".format(compound.mpid, len(competing_phases)))
    print("PMG simplex: {:.4f}".format(pmg_ehull))
    print("MP value: {:.4f}".format(mp_ehull))

In [41]:
for ipair, pair in enumerate(inconsistent_pairs):
    print("{}/{}".format(ipair+1, len(inconsistent_pairs)))
    printout_inconsistency(pair)

1/9
Compound: mp-1213897; Competing phases: 178
PMG simplex: 0.7632
MP value: 0.8855
2/9
1
Compound: mp-1318211; Competing phases: 462
PMG simplex: 10.4023
MP value: 0.0442
3/9
Compound: mp-25261; Competing phases: 202
PMG simplex: 0.0829
MP value: 0.3856
4/9
Compound: mp-647680; Competing phases: 491
PMG simplex: 0.3105
MP value: 1.2546
5/9
Compound: mp-758222; Competing phases: 907
PMG simplex: 0.0731
MP value: 0.8443
6/9
Compound: mp-763729; Competing phases: 484
PMG simplex: 0.0979
MP value: 0.4720
7/9
Compound: mp-771431; Competing phases: 240
PMG simplex: 0.1380
MP value: 0.3808
8/9
Compound: mvc-10044; Competing phases: 381
PMG simplex: 0.0593
MP value: 0.5579
9/9
Compound: mvc-15992; Competing phases: 215
PMG simplex: 0.2759
MP value: 0.5183


In [43]:
# for the 2nd pair, we can reproduce MP values by removing one compound from `competing phases`
second_pair = inconsistent_pairs[1]
compound, competing_phases = second_pair
print("MP value is: {}".format(compound.e_above_hull))
print("PMG simplex is: {}".format(find_pmgehull(*second_pair)))
for i in range(len(competing_phases)):
    new_competing_phases = competing_phases[:i] + competing_phases[i+1:]
    new_pmg_ehull = find_pmgehull(compound, new_competing_phases)
    if abs(new_pmg_ehull - compound.e_above_hull) < 1e-5:
        break
print("after removing: {}".format(competing_phases[i].mpid))
print("PMG simplex is: {}".format(new_pmg_ehull))

MP value is: 0.044174889736114054
PMG simplex is: 10.402308953305555
after removing: mp-778012
PMG simplex is: 0.0441748897361105
